In [1]:
from pyiron_base import Project
#import dolfinx
import numpy as np

2024-04-21 18:12:49,217 - pyiron_log - WARNING - pyiron found a 'templates' folder in the /home/tlakshmi/pyiron/resources resource directory. These are no longer supported in pyiron_base >=0.7.0. They are replaced by Project.create_job_class() and Project.wrap_python_function().


In [2]:
import pyiron_fenicsx

In [3]:
pr = Project('membrane_fenicsx')
pr.remove_jobs(silently=True)

0it [00:00, ?it/s]

In [4]:
job = pr.create.job.Fenicsx('mem_default')

  warnings.warn("Currently, the c++ dolfin functions used by fenicsx are not stored in the HDF5 file."



In [5]:
job.geom = job.create.geom.disk(0, 0, 0, 1, 1)
job.geom

1

In [6]:
#job.geom = job.create.geom.box(0, 0, 0, 1, 0.2, 0.2)
#job.geom

In [7]:
#job.geom = job.create.geom.rectangle(0, 0, 0, 1, 1)
#job.geom

In [8]:
#job.geom = job.create.geom.cylinder(0, 0, 0, 1, 0, 0, 0.1)
#job.geom

In [9]:
dimensions = 2
min_mesh_length = 0.03
max_mesh_length = 0.03
job.domain = job.create.domain.standardDomain(job.geom, dimensions, min_mesh_length, max_mesh_length)

Info    : Meshing 1D...
Info    : Meshing curve 1 (Ellipse)
Info    : Done meshing 1D (Wall 0.000177563s, CPU 0.000301s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.146181s, CPU 0.143121s)
Info    : 4201 nodes 8401 elements


In [10]:
#dimensions = 3
#min_mesh_length = 0.03
#max_mesh_length = 0.03
#job.domain = job.create.domain.standardDomain(job.geom, dimensions, min_mesh_length, max_mesh_length)

In [11]:
#job.domain??

In [12]:
job.mesh = job.create.get_mesh(job.domain)

In [13]:
job.V = job.create.V.functionspace(job.mesh, "Lagrange", 1)
job.V

FunctionSpace(Mesh(blocked element (Basix element (P, triangle, 1, equispaced, unset, False), (2,)), 0), Basix element (P, triangle, 1, gll_warped, unset, False))

In [14]:
job.plot.plot_mesh(job.V)

Widget(value='<iframe src="http://localhost:46437/index.html?ui=P_0x7d58101ad130_0&reconnect=auto" class="pyvi…

In [15]:
def on_boundary(x):
    return np.isclose(np.sqrt(x[0]**2 + x[1]**2), 1)

#bc1 = job.create.bc.DirichletBC_functionspace(job.V, on_boundary, 0)
bc1 = job.make_dirichletBC_functionspace(job.V, on_boundary, 0)
job.appendBC(bc1)
job.bcs

In [16]:
x = job.spatial_coord(job.mesh)
beta = job.Constant(job.mesh, 8)
R0 = job.Constant(job.mesh, 0.6)
load = 4 * job.Expression(-beta**2 * (x[0]**2 + (x[1] - R0)**2))
job.set_load(load)
#x, beta, R0, load, job.p

In [17]:
job.solvePoisson()

Coefficient(FunctionSpace(Mesh(blocked element (Basix element (P, triangle, 1, equispaced, unset, False), (2,)), 0), Basix element (P, triangle, 1, gll_warped, unset, False)), 0)

In [20]:
job.plot.plot_deformed_functionspace(job.V, job.uh, 20)

Widget(value='<iframe src="http://localhost:46437/index.html?ui=P_0x7d572bf310d0_3&reconnect=auto" class="pyvi…